# Explore here

In [63]:
# Your code here

import pandas as pd
#Importar el csv:

url= "https://raw.githubusercontent.com/4GeeksAcademy/linear-regression-project-tutorial/main/medical_insurance_cost.csv"

df= pd.read_csv(url)
#Creo 4 columnas, una para cada región y elimino la columna región 
df_dummies = pd.get_dummies(df['region'], prefix='region')
df = pd.concat([df, df_dummies], axis=1)
df.head()
df = df.drop('region', axis=1)


#Convierto los valores cualitativos en cuantitativos:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

df["sex"]= le.fit_transform(df["sex"])
df["smoker"]= le.fit_transform(df["smoker"])
df["region_northeast"]= le.fit_transform(df["region_northeast"])
df["region_northwest"]= le.fit_transform(df["region_northwest"])
df["region_southeast"]= le.fit_transform(df["region_southeast"])
df["region_southwest"]= le.fit_transform(df["region_southwest"])


#Eliminar duplicados:
df.drop_duplicates()
df.head()
#Resumen de correlación de datos
import statsmodels.api as sm

X = df.drop(["charges", "region_northeast"], axis=1)
y = df["charges"]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

df.head()




                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Wed, 21 Aug 2024   Prob (F-statistic):               0.00
Time:                        19:22:34   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.194e+04    987.819  

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


Según los datos: 
La edad aumenta en 257.2881 unidades el precio medio del seguro de salud.
El sexo p>0.1 por lo que no es relevante y la variable smoker es la que más incrementa el precio del seguro. 
El bmi aumenta en 332.5701 unidades el precio medio del seguro de salud.
El número de hijos aumenta en 479.3694 unidades el precio medio del seguro de salud.
Ser fumador aumenta en 23820 unidades el precio medio del seguro de salud.
Vivir en la región northwest supone una disminución del precio 352.9639 unidades en comparación con la región northeast
Vivir en la región southeast supone una disminución del precio 1035.0220 unidades en comparación con la región northeast
Vivir en la región southeast supone una disminución del precio 960.0510 unidades en comparación con la región northeast
Por tanto, la región northeast tiene los seguros de salud más caros.

El valor de R-cuadrado muestra que la variabilidad del precio del seguro de salud está explicado en un 75% con estas variables

Además, la región en la que vives puede disminuir el precio del seguro. Siendo 